In [1]:
import torch
import numpy as np
from torch_models import load_models, DNN, EnsembleDNN
import matplotlib.pyplot as plt
from attacks import pgd
import random
import cleverhans
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf
from cleverhans.model import CallableModelWrapper
from cleverhans.attacks import MomentumIterativeMethod
import tensorflow as tf
import torch.nn as nn
%load_ext autoreload
%autoreload 2
%matplotlib inline

/home/juanky/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
torch.cuda.empty_cache()

In [3]:
noise_budget = 0.8

In [4]:
models = load_models('imagenet')

/home/juanky/miniconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [5]:
images = torch.load('experiment_data/imagenet/imagenet_images.pt').cuda()
labels = torch.load('experiment_data/imagenet/imagenet_labels.pt').cuda()

In [6]:
[model.accuracy(images, labels, batch=True) for model in models]

[1.0, 1.0, 1.0, 1.0, 1.0]

In [7]:
ensemble = EnsembleDNN(models).eval()

In [8]:
ensemble.accuracy(images, labels, batch=True)

1.0

In [9]:
def mim_attack(model, images, noise_budget, iters):
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    input_placeholder = tf.placeholder(tf.float32, shape=(None, 3, 224, 224,))
    mim_params = {'eps': noise_budget,
                  'eps_iter': noise_budget / iters,
                  'nb_iter': iters,
                  'clip_min': 0,
                  'clip_max': 1, 
                  'ord': 2, 
                  'decay_factor': 1.0}
    tf_model = convert_pytorch_model_to_tf(model, out_dims=1000)
    cleverhans_model = CallableModelWrapper(tf_model, output_layer='logits')
    mim = MomentumIterativeMethod(cleverhans_model, sess=sess)
    adv_example_op = mim.generate(input_placeholder, **mim_params)
    perturbed_images = []
    for image in images:
        adv_example = sess.run(adv_example_op, feed_dict={input_placeholder: image.unsqueeze(0)})
        perturbed_images.append(adv_example[0])
    return np.array(perturbed_images)

### Ensemble Baseline

In [10]:
def compute_model_accs_per_point(models, images, noise_vectors, labels):
    model_accs_per_point = []
    num_points = len(images)
    for i in range(num_points):
        x = images[i].unsqueeze(0)
        v = noise_vectors[i].unsqueeze(0)
        y = labels[i]
        model_accs_per_point.append([model.accuracy(x + v, y) for model in models])
    return np.array(model_accs_per_point)

In [18]:
ensemble_baseline = mim_attack(ensemble, images.cpu(), noise_budget, 40)

In [19]:
ensemble_baseline = torch.tensor(ensemble_baseline).cuda()

In [20]:
ensemble_noise_vectors = ensemble_baseline - images

In [21]:
ensemble_accs_per_point = compute_model_accs_per_point(models, images, ensemble_noise_vectors, labels)

In [22]:
max([v.norm().item() for v in ensemble_noise_vectors])

0.788555383682251

In [23]:
ensemble_accs_per_point.shape

(100, 5)

In [24]:
print("Ensemble Baseline")
print("Mean ", np.mean(np.mean(ensemble_accs_per_point, axis=1)), 
      "Max ", np.mean(np.max(ensemble_accs_per_point, axis=1)))

Ensemble Baseline
Mean  0.262 Max  0.55


### Average Vector + Best Individual Baseline

In [25]:
individual_baselines = [mim_attack(model, images.cpu(), noise_budget, 40) for model in models]

In [26]:
individual_baselines = torch.tensor(np.array(individual_baselines)).cuda()

In [27]:
individual_baselines.shape

torch.Size([5, 100, 3, 224, 224])

In [28]:
average_vector_baseline = torch.mean(individual_baselines, dim=0)

In [29]:
average_vector_baseline.shape

torch.Size([100, 3, 224, 224])

In [30]:
average_vector_noise_vectors = average_vector_baseline - images

In [31]:
average_vector_accs_per_point = compute_model_accs_per_point(models, images,
                                                             average_vector_noise_vectors, labels)

In [32]:
print("Average Vector Baseline")
print("Mean ", np.mean(np.mean(average_vector_accs_per_point, axis=1)), 
      "Max ", np.mean(np.max(average_vector_accs_per_point, axis=1)))

Average Vector Baseline
Mean  0.5439999999999999 Max  0.84


In [33]:
individual_noise_vectors = torch.stack([individual_baselines[i] - images for i in range(len(models))])

In [34]:
[max([v.norm().item() for v in t]) for t in individual_noise_vectors]

[0.7824881672859192,
 0.7879824042320251,
 0.7826390862464905,
 0.771580696105957,
 0.7906122207641602]

In [35]:
def compute_best_individual_baseline(models, images, individual_noise_vectors, labels):
    individual_accs_per_model = np.array([compute_model_accs_per_point(models, images, 
                                                                       individual_noise_vectors[i], labels)
                                        for i in range(len(models))])
    mean_max = []
    for summary in [np.mean, np.max]:
        overall = []
        for i in range(len(images)):
            point_accuracy_per_model = []
            for m in range(len(models)):
                r = summary(individual_accs_per_model[m][i])
                point_accuracy_per_model.append(r)
            overall.append(min(point_accuracy_per_model))
        mean_max.append(np.mean(overall))
    return mean_max

In [36]:
print("Best Individual Baseline")
print("Mean Max", compute_best_individual_baseline(models, images, individual_noise_vectors, labels))

Best Individual Baseline
Mean Max [0.7019999999999998, 0.99]


In [ ]:
exp_dir = 'experiment_results/imagenet_1/'
noise_vectors = torch.load(exp_dir + 'noise_vectors.pt')
expected_losses = np.load(exp_dir + 'expected_losses.npy')
minimum_losses = np.load(exp_dir + 'minimum_losses.npy')

In [ ]:
num_points = 100
mwu_iters = 30
model_accs_per_point = []
for i in range(num_points):
    x = images[i].unsqueeze(0)
    y = labels[i]
    model_accs = []
    for t in range(mwu_iters):
        v = noise_vectors[t][i].cuda()
        model_accs.append([model.accuracy(x + v, y) for model in models])
    model_accs_per_point.append(np.array(model_accs))

In [ ]:
max_acc_plot = []
mean_acc_plot = []
for t in range(mwu_iters):
    mean_acc = np.mean([np.mean(np.mean(model_accs[:t+1], axis=0)) for model_accs in model_accs_per_point])
    max_acc = np.mean([np.max(np.mean(model_accs[:t+1], axis=0)) for model_accs in model_accs_per_point])
    max_acc_plot.append(max_acc)
    mean_acc_plot.append(mean_acc)

In [ ]:
max_acc_plot[-1], mean_acc_plot[-1]

In [ ]:
plt.plot(max_acc_plot, color='orange')
plt.xlabel('MWU Iterations')
plt.ylabel('Max Model Accuracy')
plt.title('ImageNet')
# plt.savefig('deep.pdf')

### Visual Comparisons

In [ ]:
from torchvision.transforms import ToPILImage

In [ ]:
i=17
ToPILImage(mode='RGB')(images[i].cpu())

In [ ]:
(ensemble_baseline[i] - images[i]).norm()

In [ ]:
ToPILImage(mode='RGB')(ensemble_baseline[i].cpu())

In [ ]:
perturb_im = ToPILImage(mode='RGB')(images[i].cpu() + noise_vectors[0][i][0].cpu())
im.save('im_{}.png'.format(i))
perturb_im.save('perturb_im_{}.png'.format(i))